In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity


dataset = pd.read_csv("kinopoisk-top250.csv")

dataset = dataset.drop(columns=['url_logo', 'rating', 'country', 'rating_ball', 'director', 'actors', 'year'])

dataset

,movie,overview,screenwriter
0,Побег из Шоушенка,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...,Фрэнк Дарабонт; Стивен Кинг
1,Зеленая миля,Пол Эджкомб — начальник блока смертников в тюр...,Фрэнк Дарабонт; Стивен Кинг
2,Форрест Гамп,От лица главного героя Форреста Гампа; слабоум...,Эрик Рот; Уинстон Грум
3,Список Шиндлера,Фильм рассказывает реальную историю загадочног...,Стивен Зеллиан; Томас Кенилли
4,1+1,Пострадав в результате несчастного случая; бог...,Оливье Накаш; Филипп Поццо ди Борго; Эрик Т...
...,...,...,...
245,Крамер против Крамера,Тед Крамер; целиком посвятив себя карьере; не ...,Роберт Бентон; Эйвери Кормэн
246,Пираты Карибского моря: На краю Света,Новые приключения Джека Воробья и его друзей У...,Тед Эллиот; Терри Россио; Стюарт Битти; ...
247,Иди и смотри,Флера — шестнадцатилетний мальчишка; откопавши...,Алесь Адамович; Элем Климов
248,Профессионал,Убить президента африканской республики. Такое...,Мишель Одиар; Патрик Александер; Жорж Лотне...


In [2]:
# Объединяем все текстовые поля в одно
df = dataset.copy()
id_column = 'movie'
columns_to_merge = df.columns.drop(id_column)

df['full_text'] = df[columns_to_merge].astype(str).agg(' '.join, axis=1)

# Наш рабочий датафрейм
final_df = df[[id_column, 'full_text']].copy()
print("Подготовленный DataFrame:")
print(final_df)

Подготовленный DataFrame:
                                      movie  \
0                        Побег из Шоушенка    
1                             Зеленая миля    
2                             Форрест Гамп    
3                          Список Шиндлера    
4                                      1+1    
..                                      ...   
245                  Крамер против Крамера    
246  Пираты Карибского моря: На краю Света    
247                           Иди и смотри    
248                           Профессионал    
249    Холодное лето пятьдесят третьего...    

                                             full_text  
0    Бухгалтер Энди Дюфрейн обвинён в убийстве собс...  
1    Пол Эджкомб — начальник блока смертников в тюр...  
2    От лица главного героя Форреста Гампа; слабоум...  
3    Фильм рассказывает реальную историю загадочног...  
4    Пострадав в результате несчастного случая; бог...  
..                                                 ...  
245  Тед К

In [3]:
# 1. Инициализация векторизатора
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,4))


# 2. Обучение на столбце 'full_text' и преобразование данных в матрицу
tfidf_matrix = vectorizer.fit_transform(final_df['full_text'])
print(vectorizer.get_feature_names_out())


['000' '000 золотых' '000 золотых рублей' ... 'ящик отправляются'
 'ящик отправляются кругосветное'
 'ящик отправляются кругосветное путешествие']


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

while True:
    user_query = input()
    
    if user_query.lower == 'выход':
        break

    # 2. Преобразование запроса в TF-IDF вектор. 
    # ВАЖНО: используем метод .transform(), а не .fit_transform(), 
    # чтобы применить словарь и веса, уже посчитанные на наших данных.
    query_vector = vectorizer.transform([user_query])

    # 3. Вычисление косинусного сходства
    # Сравниваем вектор запроса со всеми векторами из нашей матрицы документов
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)

    # Результат — это матрица, где для каждой строки запроса (у нас одна)
    # есть сходство с каждым документом. Нам нужна первая строка.
    similarity_scores = cosine_similarities[0]

    # 4. Нахождение индекса документа с максимальным сходством
    best_match_index = np.argmax(similarity_scores)

    # 5. Получение title и текста лучшего совпадения
    best_match_title = final_df.loc[best_match_index, 'movie']
    best_match_score = similarity_scores[best_match_index]

    print(f"\nЗапрос пользователя: '{user_query}'")
    print(f"Наиболее подходящий товар: '{best_match_title}'")
    print(f"Степень сходства: {best_match_score:.4f}")

KeyboardInterrupt: Interrupted by user